# Sample Questions for the Chatbot

## Can I use the same color twice? 
## Give me tips to play better? 
## What is the goal? 
## How to play this game?

In [ ]:

# Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
import pickle
from logic import *
import speech_recognition as sr
import pyttsx3
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random



In [ ]:

# Load dataset (make sure the CSV is in the same folder as this notebook)
df = pd.read_csv(r"Data_mastermind_chatbot_dataset.csv")
df.head()


In [ ]:


# Encode the intent labels
label_encoder = LabelEncoder() #creates a label encoder object from sklearn.preprocessing. You use it to convert categorical labels (one word or text) into numeric values
y = label_encoder.fit_transform(df["intent"]) #This line uses the LabelEncoder to convert the values in the intent column of your DataFrame into numbers, and stores the result in y. 

# Tokenize the text queries
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>") #This creates a Tokenizer object from Keras (in tensorflow.keras.preprocessing.text). It's used to convert text (words) into sequences of numbers, which machine learning models (especially neural networks) can understand. Only keep the top 2,000 most frequent words from your dataset. Other words are ignored. Any word not in the top 2,000 will be replaced with "<OOV>", which stands for "Out-Of-Vocabulary" 

tokenizer.fit_on_texts(df["query"]) #Scan all the sentences (more than one word) in df["query"] and learn which words exist and how often they appear. 'hello’: 2,     'how’: 3,     'are’: 4,
X = tokenizer.texts_to_sequences(df["query"]) #: Each word in the sentence is replaced by its corresponding integer index from the tokenizer’s vocabulary (word_index). {'<OOV>': 1, 'hello': 2, 'how': 3, 'are': 4, 'you': 5, 'tell': 6, 'me': 7

# Pad the sequences
X_padded = pad_sequences(X, maxlen=20, padding='post', truncating='post') #This uses pad_sequences() from tensorflow.keras.preprocessing.sequence to make all the input sequences the same length.  20 tokens length, padding = post  Add zeros at the end of short sequences, truncating = post  Cut extra tokens from the end of long sequences


In [ ]:

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [ ]:

# Build and compile the ANN model
model = Sequential([
    Embedding(input_dim=2000, output_dim=16, input_length=20),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


In [ ]:
# === STEP 6: Save Everything ===
model.save(r"chatbot_model.h5")

with open(r"tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

with open(r"label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

In [ ]:
def plot_color_sequence(solution):
    """
    Plot 4 colored circles representing the correct color sequence.
    """
    fig, ax = plt.subplots()
    ax.set_xlim(0, 5)
    ax.set_ylim(0, 2)
    ax.set_aspect('equal')
    ax.axis('off')  # Hide axis

    for i, color in enumerate(solution):
        circle = plt.Circle((i + 1, 1), 0.4, color=color.lower())
        ax.add_patch(circle)
        ax.text(i + 1, 0.4, color.capitalize(), ha='center', fontsize=10)

    plt.title("\n🎯 Correct Color Sequence")
    plt.show()

In [ ]:
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [ ]:
def get_chatbot_response(query=None, tokenizer=None, model=None, label_encoder=None, max_len=20, use_voice=False):
    """
    Get chatbot response using text or voice input, and return output via text or voice.

    Parameters:
        query (str): User input text. Ignored if use_voice=True.
        tokenizer: Trained tokenizer object.
        model: Trained classification model.
        label_encoder: Trained label encoder.
        max_len (int): Maximum sequence length for padding.
        use_voice (bool): Whether to use voice input/output.

    Returns:
        str: Chatbot response
    """
####################################
    responses = {
        'color_repeat': "Yes, you can use the same color more than once in your guess.",
        'feedback_meaning': "Black pegs mean right color in the right position. White pegs mean right color in the wrong position.",
        'game_strategy': "Start with a guess of different colors. Use the feedback to eliminate possibilities.",
        'restart_game': "Game restarted. Try to crack the new code!",
        'winning_condition': "You win by guessing the exact color sequence in the correct order.",
        'quit_game': "Thanks for playing Mastermind. Goodbye!"
    }
##################################
    # Voice input (if enabled)
    if use_voice:

        recognizer = sr.Recognizer()
        # Retry up to 3 times if voice fails
        for attempt in range(3):
                with sr.Microphone() as source:
                    speak("What would you like to ask? I am listening")
                    print("🔊 Listening...")
                    audio = recognizer.listen(source)
                    try:
                        query = recognizer.recognize_google(audio)
                        print(f"🗣️ You said: {query}")
                        speak(f"You said: {query}")
                        break
                    except sr.UnknownValueError:
                        speak("Sorry, I didn't catch that. ask your question again.")
                    except sr.RequestError:
                        error_message = "Speech service is unavailable."
                        speak(error_message)
                        return

    # Tokenize and pad the input
    seq = tokenizer.texts_to_sequences([query])
    padded = pad_sequences(seq, maxlen=max_len, padding='post', truncating='post')

    # Predict intent
    prediction = model.predict(padded)
    intent_index = np.argmax(prediction)
    predicted_intent = label_encoder.inverse_transform([intent_index])[0]

    # Get response
    response = responses.get(predicted_intent, "I'm not sure how to respond to that.")

    # Voice output (if enabled)
    #if use_voice:
        #speak(response)
    return response


In [ ]:
def run_chatbot(tokenizer, model, label_encoder):
    """
    Interact with the chatbot using voice or text.
    After each answer, ask if the user wants to continue chatting or play the Mastermind game.
    Includes voice input/output with retry for errors.
    """

    # Voice welcome and choice
    speak("Welcome to the Mastermind Chatbot.")
    speak("Would you like to use voice interaction? Type yes or no in the prompt below")
    mode = input("Would you like to use voice interaction? (yes/no): ").strip().lower()
    if mode=="yes":
        speak("Ok, Voice mode activated! I'm all ears")
    max_attempts = 3

    while True:
        if mode == "yes":
            
            #speak("What would you like to ask?")
            #response = get_chatbot_response(tokenizer=tokenizer, model=model, label_encoder=label_encoder, use_voice=True)

            # Ask user what's next
            speak("What's next? You can ask questions, jump into the Mastermind game, or exit the chat. Just say: question, game, or exit!")

            recognizer = sr.Recognizer()

            # Retry up to 3 times if voice fails
            for attempt in range(max_attempts):
                remaining = max_attempts - attempt
                print(attempt)
                with sr.Microphone() as source:
                    print("🔊 I am listening, please speak clearly...")
                    audio = recognizer.listen(source, timeout=10, phrase_time_limit=15)
                    try:
                    
                        query = recognizer.recognize_google(audio)
                        if query.lower() in ["question", "game", "exit"]:
                            print(f"🗣️ You said: {query}")
                            speak(f"You said: {query}")
                            break
                        else:
                            if remaining >= 1:
                                speak(f"Sorry, Wrong response. You have {remaining - 1} more attempt{'s' if remaining - 1 > 1 else ''}.")
                                speak("Just say: question, game, or exit clearly!")
                            else:
                                speak("Sorry, Invalid response. Let's try again later.")
                    except sr.UnknownValueError:
                        if remaining >= 1:
                                speak(f"Sorry, I didn't catch that. You have {remaining - 1} more attempt{'s' if remaining - 1 > 1 else ''}.")
                                speak("Just say: question, game, or exit clearly!")
                        else:
                                speak("Sorry, I still couldn't hear you. Let's try again later.")
                    except sr.RequestError:
                        error_message = "Speech service is unavailable."
                        speak(error_message)
                        return
                   

            
            if query.lower() == "question":
                response = get_chatbot_response(tokenizer=tokenizer, model=model, label_encoder=label_encoder, use_voice=True)
                speak(response)
            elif query.lower() == "game":
                speak("🎮 Launching the Mastermind game...")
                play_mastermind()
                break
            elif query.lower() == "exit":
                speak("👋 Goodbye!")
                break
            else:
                break
        else:
            query = input("💬 What's next? You can ask questions, jump into the Mastermind game, or exit the chat. Just say: question, game, or exit!")
            if query.lower() == "question":
                response = get_chatbot_response(tokenizer=tokenizer, model=model, label_encoder=label_encoder, use_voice=True)
                print("🤖 Chatbot:", response)
            elif query.lower() == "game":
                print("🎮 Launching the Mastermind game...")
                play_mastermind()
                break
            elif query.lower() == "exit":
                print("👋 Goodbye!")
                break
            else:
                print("This is not a valied response. Let's try again later.")
                break
 

In [ ]:
def play_mastermind():
    """
    Mastermind with auto-feedback:
      - Randomly selects 4 distinct colors for the secret (order hidden).
      - Displays the set of allowed colors in random order (not the secret sequence).
      - Prompts player for guesses like 'red0', 'blue1', etc.
      - Function automatically computes number of correct positions.
    """
    # ---------- 1) Choose allowed colors and secret ----------
    palette = ["red", "blue", "green", "yellow", "orange", "purple", "pink", "brown", "cyan", "magenta"]
    allowed_colors = random.sample(palette, 4)          # choose 4 distinct colors
    secret_order = random.sample(allowed_colors, 4)     # shuffle them into a hidden solution
    secret_solution = {i: secret_order[i] for i in range(4)}

    # Display allowed colors in a randomized order (not the true sequence)
    print("Valid colors for this round (secret order is hidden):", ", ".join(random.sample(allowed_colors, len(allowed_colors))))
    

    # ---------- 2) Build symbols and base constraints ----------
    symbols = [Symbol(f"{color}{i}") for i in range(4) for color in allowed_colors]
    knowledge = And()

    # Each color appears in at least one position
    for color in allowed_colors:
        knowledge.add(Or(*(Symbol(f"{color}{i}") for i in range(4))))

    # A color cannot occupy two different positions
    for color in allowed_colors:
        for i in range(4):
            for j in range(4):
                if i != j:
                    knowledge.add(Implication(
                        Symbol(f"{color}{i}"), Not(Symbol(f"{color}{j}"))
                    ))

    # Each position has at least one color
    for i in range(4):
        knowledge.add(Or(*(Symbol(f"{c}{i}") for c in allowed_colors)))

    # Mutual exclusion per position
    for i in range(4):
        for c1 in allowed_colors:
            for c2 in allowed_colors:
                if c1 != c2:
                    knowledge.add(Implication(
                        Symbol(f"{c1}{i}"), Not(Symbol(f"{c2}{i}"))
                    ))

    # ---------- 3) Gameplay loop ----------
    def parse_guess(token: str):
        if not token:
            return None, None
        color, pos_char = token[:-1], token[-1]
        if not pos_char.isdigit():
            return None, None
        pos = int(pos_char)
        if color not in allowed_colors or pos not in (0, 1, 2, 3):
            return None, None
        return color, pos

    number_trials = int(input('Number of trials: ').strip())

    for trial in range(number_trials):
        print(f"\n--- Trial {trial + 1} of {number_trials} ---")
        x0 = input('Input first color as ###0: ').strip()
        x1 = input('Input second color as ###1: ').strip()
        x2 = input('Input third color as ###2: ').strip()
        x3 = input('Input fourth color as ###3: ').strip()
        guess_tokens = [x0, x1, x2, x3]

        parsed = [parse_guess(t) for t in guess_tokens]
        if any(c is None for c, _ in parsed):
            print("\nInvalid guess. Please try again with the displayed colors and positions 0..3.\n")
            continue

        # ---------- Auto-check ----------
        number_correct = sum(1 for (c, p) in parsed if secret_solution.get(p) == c)
        print(f"System feedback: {number_correct} correct position(s).")

        # ---------- Update KB ----------
        if number_correct == 0:
            knowledge.add(And(*(Not(Symbol(tok)) for tok in guess_tokens)))
        elif number_correct == 1:
            knowledge.add(Or(*(Symbol(tok) for tok in guess_tokens)))
        elif number_correct == 2:
            knowledge.add(Or(
                And(Symbol(guess_tokens[0]), Symbol(guess_tokens[1]), Not(Symbol(guess_tokens[2])), Not(Symbol(guess_tokens[3]))),
                And(Symbol(guess_tokens[0]), Symbol(guess_tokens[2]), Not(Symbol(guess_tokens[1])), Not(Symbol(guess_tokens[3]))),
                And(Symbol(guess_tokens[0]), Symbol(guess_tokens[3]), Not(Symbol(guess_tokens[1])), Not(Symbol(guess_tokens[2]))),
                And(Symbol(guess_tokens[1]), Symbol(guess_tokens[2]), Not(Symbol(guess_tokens[0])), Not(Symbol(guess_tokens[3]))),
                And(Symbol(guess_tokens[1]), Symbol(guess_tokens[3]), Not(Symbol(guess_tokens[0])), Not(Symbol(guess_tokens[2]))),
                And(Symbol(guess_tokens[2]), Symbol(guess_tokens[3]), Not(Symbol(guess_tokens[0])), Not(Symbol(guess_tokens[1])))
            ))
        elif number_correct >= 3:
            knowledge.add(And(*(Symbol(tok) for tok in guess_tokens)))

        # ---------- Try to infer solution ----------
        inferred_colors, entailed_count = [], 0
        for sym in symbols:
            if model_check(knowledge, sym):
                print("Entailed:", sym)
                entailed_count += 1
                inferred_colors.append(sym.name[:-1])

        if entailed_count == 4:
            print('...Solved...')
            try:
                speak("The system nailed it!")
            except Exception:
                pass
            try:
                plot_color_sequence(inferred_colors)
            except Exception:
                pass
            break

    print("\nSecret solution (position -> color):", secret_solution)


In [ ]:
#Driver Code
run_chatbot(tokenizer, model, label_encoder)
